<a href="https://colab.research.google.com/github/yuliant/sentimen_analisis_jne_knn/blob/main/sentimen_analisis_twitter_knn_part2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

In [3]:
#periksa data dan urutkan berdasarkan tanggal
csv = pd.read_csv('data_search.csv')
csv.sort_values(by="Tanggal", ascending=False).head(5)

,Unnamed: 0,Tanggal,Pengguna,Tweet,Sentimen
0,0,2023-02-14 14:34:29,ktaehyun__g,_ID tolong cek dm min,Netral
1,1,2023-02-14 14:32:53,sweebypie,ini aku 11kg karena kena volume. harusnya 8kg ...,Netral
2,2,2023-02-14 14:25:21,doyok__31,_ID Kapan sampe nya ke rumah saya ya,Netral
3,3,2023-02-14 14:24:32,kafie_ribedz,"Hai admin, masih di ngirim paket di jne rawaku...",Netral
4,4,2023-02-14 14:16:59,asiaticakjw,jne cargo emg mahal nder klo mau yg murah cb p...,Netral


# PROSES 2 - FILTERING

In [4]:
!pip install sastrawi

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 KB 7.7 MB/s eta 0:00:00


In [5]:
!pip install vaderSentiment

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 KB 6.5 MB/s eta 0:00:00


In [6]:
import matplotlib.pyplot as plt
import re
import string
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [7]:
csv.shape

(100, 5)

In [9]:
#menambahkan kolom baru yaitu "label"
#Netral = 0
#Positif = 1
#Negatif = 2

sentimen = []
for index, row in csv.iterrows():
  if row['Sentimen'] == 'Netral':
    sentimen.append('0')
  elif row['Sentimen'] == 'Positif':
    sentimen.append('1')
  else:
    sentimen.append('2')

csv['label'] = sentimen
csv.head(15)


,Unnamed: 0,Tanggal,Pengguna,Tweet,Sentimen,label
0,0,2023-02-14 14:34:29,ktaehyun__g,_ID tolong cek dm min,Netral,0
1,1,2023-02-14 14:32:53,sweebypie,ini aku 11kg karena kena volume. harusnya 8kg ...,Netral,0
2,2,2023-02-14 14:25:21,doyok__31,_ID Kapan sampe nya ke rumah saya ya,Netral,0
3,3,2023-02-14 14:24:32,kafie_ribedz,"Hai admin, masih di ngirim paket di jne rawaku...",Netral,0
4,4,2023-02-14 14:16:59,asiaticakjw,jne cargo emg mahal nder klo mau yg murah cb p...,Netral,0
5,5,2023-02-14 14:05:57,JNECare,"Sama-sama kak, Terima kasih telah menggunakan ...",Positif,1
6,6,2023-02-14 14:04:31,FicoFey,"Iya e, akhire kyknya tetep pake JNE ato pos kilat",Netral,0
7,7,2023-02-14 14:02:56,hmmitha,jne ga jls anjir,Netral,0
8,8,2023-02-14 14:02:53,JNE_RPW,FORMAT! #JNEConfess for : -uname tujuan- -isi ...,Netral,0
9,9,2023-02-14 13:57:56,Trililililily,"Nmax diserempet mobil box jne, untung kaga tum...",Netral,0


In [10]:
csv.shape

(100, 6)

In [11]:
#menghilangkan kolom yang tidak dipakai

csv_data = csv.copy()
csv_data = csv.drop(columns=['Tanggal', 'Pengguna', 'Sentimen'])
csv_data.head()

,Unnamed: 0,Tweet,label
0,0,_ID tolong cek dm min,0
1,1,ini aku 11kg karena kena volume. harusnya 8kg ...,0
2,2,_ID Kapan sampe nya ke rumah saya ya,0
3,3,"Hai admin, masih di ngirim paket di jne rawaku...",0
4,4,jne cargo emg mahal nder klo mau yg murah cb p...,0


In [13]:
csv_data.shape

(100, 3)

In [22]:
x = csv_data.iloc[:, 1].values
y = csv_data.iloc[:, 2].values

In [23]:
#mengambil 20% data untuk data test
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=0)

In [24]:
x_train

array(['Paket saya juga sudah 5 hari belum jalan. Masih di daerah asal pengiriman statusnya. . Makin lama jne makin ga layak di pertahankan.',
       'Oke, kirim nya pake JNE ya',
       'Hai admin, masih di ngirim paket di jne rawakuning? Hehehe',
       'Punya ol shop? Tp suka males antri kirim paket? Kami solusinya. Spesialis jemput paket via JNE di Kota Depok , WA/call di 0856 8000027',
       '_ID Kelanjutannya dikenalin sm mas2 yg kerja di madukoro wkwk pake orang dalem deh akhirnya, sad but true',
       'Aneh ya, gak masuk akal 😁. Baru baru ini aja begitu, dulunya enggak..sesuai di web jne lebih mahal dikit lah masih masuk logika..wajar kalau diup dikit. Tapi kalau sekarang gak masuk akal, cuma di aja bisa gitu 🤭🤭🤭. Tq min, mungkin di suruh pindah ke ekspedisi lain.',
       'Iya e, akhire kyknya tetep pake JNE ato pos kilat',
       'Hai kak, akun Tokopedia Seller saya diretas sejak Jumat 10 Februari 2023 lalu Si penyusup sudah menarik Saldo Tokopedia sebesar sekitar Rp. 8 jut

In [25]:
len(x_train)

80

In [26]:
len(x)

100

In [27]:
x_test

array(['MANA BISA DIKASIH SEGEL FRAGIL. DIKIRA PAKET JNE',
       'Sangat tidak disarankan memang pakai jasa Next Day _ID 😅😅😅',
       '_ID Kapan sampe nya ke rumah saya ya',
       'Bila tim pick up JNE Abadi Sejahtera kurang berkenan atau kurang sopan dalam pelayanannya bisa mention kami utk ditindak lanjuti,terima kasih',
       'di bandungs aya kedai jiga warpopskay teu yes, jne euyyy',
       'halo _ID, jne ada jasa packing barang ga ya? misalnya packing gitar pake kardus gitar + kayu.',
       '_chnle jne dongs',
       'Awalnya Rp 6.049.000 dapet potongan 400k jadi Rp 5.762.080 Ini udah include Biaya Asuransi (Meminimalisir kalo terjadi apa2) dan Include Biaya Pengiriman Jakarta ke Surabaya (Kirim barang elektronik pake JNE aja) Bisaa juga Checkout Packaging Kayu 30k biar lebih aman',
       'Seru amat ambil almet trs foto", yg almetnya dipaketin jne could never (soalnya pas oplen dah abis hype nya 👎🏻)',
       '_ID Min cek dm',
       'Petugas Cash Counter kami akan selalu mela

In [28]:
len(x_test)

20

In [29]:
#preprocessing

nltk.download('punkt')
nltk.download('stopword')
nltk.download('wordnet') #stemming

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Error loading stopword: Package 'stopword' not found in
[nltk_data]     index
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [32]:
csv_data['Tweet']

0                                 _ID tolong cek dm min
1     ini aku 11kg karena kena volume. harusnya 8kg ...
2                  _ID Kapan sampe nya ke rumah saya ya
3     Hai admin, masih di ngirim paket di jne rawaku...
4     jne cargo emg mahal nder klo mau yg murah cb p...
                            ...                        
95                                       _ID Min cek dm
96    #TemanJNE sudah kirim hadiah untuk orang tersa...
97    WTS makeup dan skincare last stock yg masih ba...
98    WTS makeup dan skincare last stock yg masih ba...
99    Hai kak, akun Tokopedia Seller saya diretas se...
Name: Tweet, Length: 100, dtype: object

In [38]:
#cleaning
#remove url
csv_data['Tweet'] = csv_data['Tweet'].str.replace('https\S',' ',case=False)

#merubah teks menjadi huruf kecil
csv_data['Tweet'] = csv_data['Tweet'].str.lower()

#remove mention
csv_data['Tweet'] = csv_data['Tweet'].str.replace('@\S+',' ',case=False)

#remove hastag
csv_data['Tweet'] = csv_data['Tweet'].str.replace('#\S+',' ',case=False)

#remove next character
csv_data['Tweet'] = csv_data['Tweet'].str.replace("\'\w+",' ',case=False)

#remove punctuation
csv_data['Tweet'] = csv_data['Tweet'].str.replace(r'[^\w\s]',' ',case=False)

#remove underscore
csv_data['Tweet'] = csv_data['Tweet'].str.replace(r'[\W_]',' ',case=False)

#remove number
csv_data['Tweet'] = csv_data['Tweet'].str.replace(r'\w*\d+\w*',' ',case=False)

#remove spasi berlebih
csv_data['Tweet'] = csv_data['Tweet'].str.replace('\s(2)',' ',case=False)

<ipython-input-38-6815028165d3>:3: FutureWarning: The default value of regex will change from True to False in a future version.
  csv_data['Tweet'] = csv_data['Tweet'].str.replace('https\S',' ',case=False)
<ipython-input-38-6815028165d3>:9: FutureWarning: The default value of regex will change from True to False in a future version.
  csv_data['Tweet'] = csv_data['Tweet'].str.replace('@\S+',' ',case=False)
<ipython-input-38-6815028165d3>:12: FutureWarning: The default value of regex will change from True to False in a future version.
  csv_data['Tweet'] = csv_data['Tweet'].str.replace('#\S+',' ',case=False)
<ipython-input-38-6815028165d3>:15: FutureWarning: The default value of regex will change from True to False in a future version.
  csv_data['Tweet'] = csv_data['Tweet'].str.replace("\'\w+",' ',case=False)
<ipython-input-38-6815028165d3>:18: FutureWarning: The default value of regex will change from True to False in a future version.
  csv_data['Tweet'] = csv_data['Tweet'].str.repl

In [39]:
csv_data['Tweet']

0                                  id tolong cek dm min
1     ini aku   karena kena volume  harusnya   kalau...
2                   id kapan sampe nya ke rumah saya ya
3     hai admin  masih di ngirim paket di jne rawaku...
4     jne cargo emg mahal nder klo mau yg murah cb p...
                            ...                        
95                                        id min cek dm
96      sudah kirim hadiah untuk orang tersayang har...
97    wts makeup dan skincare last stock yg masih ba...
98    wts makeup dan skincare last stock yg masih ba...
99    hai kak  akun tokopedia seller saya diretas se...
Name: Tweet, Length: 100, dtype: object

In [41]:
#tokenizing

from nltk.tokenize import word_tokenize

x = csv_data.iloc[0]
print(nltk.word_tokenize(x['Tweet']))

['id', 'tolong', 'cek', 'dm', 'min']


In [42]:
def identify_tekens(row) :
  text = row['Tweet']
  tokens = nltk.word_tokenize(text)
  token_words = [w for w in tokens if w.isalpha()]
  return token_words

csv_data['Tweet'] = csv_data.apply(identify_tekens, axis=1)
csv_data.Tweet

0                            [id, tolong, cek, dm, min]
1     [ini, aku, karena, kena, volume, harusnya, kal...
2          [id, kapan, sampe, nya, ke, rumah, saya, ya]
3     [hai, admin, masih, di, ngirim, paket, di, jne...
4     [jne, cargo, emg, mahal, nder, klo, mau, yg, m...
                            ...                        
95                                   [id, min, cek, dm]
96    [sudah, kirim, hadiah, untuk, orang, tersayang...
97    [wts, makeup, dan, skincare, last, stock, yg, ...
98    [wts, makeup, dan, skincare, last, stock, yg, ...
99    [hai, kak, akun, tokopedia, seller, saya, dire...
Name: Tweet, Length: 100, dtype: object

In [43]:
#stemming (pembentukan kata dasar)
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
stemming = PorterStemmer()

In [44]:
def stem_list(row) :
  text = row['Tweet']
  stem = [stemming.stem(word) for word in text]
  return(stem)

csv_data['Tweet'] = csv_data.apply(stem_list, axis=1)
csv_data.Tweet

0                            [id, tolong, cek, dm, min]
1     [ini, aku, karena, kena, volum, harusnya, kala...
2           [id, kapan, samp, nya, ke, rumah, saya, ya]
3     [hai, admin, masih, di, ngirim, paket, di, jne...
4     [jne, cargo, emg, mahal, nder, klo, mau, yg, m...
                            ...                        
95                                   [id, min, cek, dm]
96    [sudah, kirim, hadiah, untuk, orang, tersayang...
97    [wt, makeup, dan, skincar, last, stock, yg, ma...
98    [wt, makeup, dan, skincar, last, stock, yg, ma...
99    [hai, kak, akun, tokopedia, seller, saya, dire...
Name: Tweet, Length: 100, dtype: object

In [45]:
#stopword (menghapus kata yang tidak penting)
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')

stops = set(stopwords.words('indonesian'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [47]:
csv_data.head(15)

,Unnamed: 0,Tweet,label
0,0,"[id, tolong, cek, dm, min]",0
1,1,"[ini, aku, karena, kena, volum, harusnya, kala...",0
2,2,"[id, kapan, samp, nya, ke, rumah, saya, ya]",0
3,3,"[hai, admin, masih, di, ngirim, paket, di, jne...",0
4,4,"[jne, cargo, emg, mahal, nder, klo, mau, yg, m...",0
5,5,"[sama, sama, kak, terima, kasih, telah, menggu...",1
6,6,"[iya, e, akhir, kyknya, tetep, pake, jne, ato,...",0
7,7,"[jne, ga, jl, anjir]",0
8,8,"[format, for, unam, tujuan, isi, mention, conf...",0
9,9,"[nmax, diserempet, mobil, box, jne, untung, ka...",0


In [48]:
csv_data['label'].value_counts()

0    87
1    10
2     3
Name: label, dtype: int64

#PROSES 3 - TERM WEIGTHING

In [84]:
#menghitung vector
def List_function(n):
  list_array = np.arange(1,n+1)
  return(list_array)

from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd

tfidfvectorizer = CountVectorizer(analyzer='word')
tfidf_wm = tfidfvectorizer.fit_transform(csv['Tweet'])
tfidf_tokens = tfidfvectorizer.get_feature_names()
df_tfidfvect = pd.DataFrame(data = tfidf_wm.toarray(), index=List_function(100))
print("\nTF-IDF Vectorizer\n")
print(df_tfidfvect)



TF-IDF Vectorizer

     0    1    2    3    4    5    6    7    8    9    ...  874  875  876  \
1      0    0    0    0    0    0    0    0    0    0  ...    0    0    0   
2      0    0    0    0    0    0    0    0    0    0  ...    0    0    0   
3      0    0    0    0    0    0    0    0    0    0  ...    0    0    0   
4      0    0    0    0    0    0    0    0    0    0  ...    0    0    0   
5      0    0    0    0    0    0    0    0    0    0  ...    0    0    1   
..   ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...   
96     0    0    0    0    0    0    0    0    0    0  ...    0    0    0   
97     0    0    0    0    0    0    0    0    0    0  ...    0    0    0   
98     0    0    0    0    0    0    0    0    0    0  ...    0    0    1   
99     0    0    0    0    0    0    0    0    0    0  ...    0    0    1   
100    0    0    0    0    0    0    0    0    0    0  ...    0    0    0   

     877  878  879  880  881  882  883  
1      0    0 

/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [85]:
#mengubah menjadi vector term

from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer()
X_train_cv = cv.fit_transform(x_train)
X_test_cv = cv.transform(x_test)

In [87]:
X_test_cv[1:1]

<0x752 sparse matrix of type '<class 'numpy.int64'>'
	with 0 stored elements in Compressed Sparse Row format>

In [88]:
#menggunakan perhitungan TF-IDF
from sklearn.model_selection import train_test_split

x = csv_data['label']
y = csv_data['label']

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

In [89]:
x_train

90    0
1     0
5     1
89    0
0     0
     ..
87    0
7     0
64    0
39    0
86    0
Name: label, Length: 80, dtype: object

In [90]:
len(x_train)

80

In [91]:
x_test

33    1
12    0
37    0
41    0
79    1
66    0
48    0
10    1
92    1
59    0
23    0
62    0
67    0
60    0
95    0
47    0
63    0
40    0
70    0
76    1
Name: label, dtype: object

In [92]:
len(x_test)

20

#PROSES 4 - KLASIFIKASI ALGORITMA K-NN

In [93]:
#pembuatan model KNN
from sklearn.neighbors import KNeighborsClassifier
classifier = KNeighborsClassifier(n_neighbors=3)
classifier.fit(X_train_cv, y_train)

KNeighborsClassifier(n_neighbors=3)

In [94]:
#prediksi menggunakan data test
y_pred = classifier.predict(X_test_cv)
print(y_pred)

['0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0'
 '0' '0']


In [97]:
from sklearn.metrics import classification_report, confusion_matrix
cm = confusion_matrix(y_test, y_pred)
cr = classification_report(y_test, y_pred)
print(cm)
print(cr)

[[15  0]
 [ 5  0]]
              precision    recall  f1-score   support

           0       0.75      1.00      0.86        15
           1       0.00      0.00      0.00         5

    accuracy                           0.75        20
   macro avg       0.38      0.50      0.43        20
weighted avg       0.56      0.75      0.64        20



/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
